In [181]:
!pip install feedparser beautifulsoup4 requests python-dotenv langchain-groq chromadb  datetime feedparser python-telegram-bot schedule nest-asyncio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
from dotenv import load_dotenv
import os

load_dotenv()

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    api_key=os.getenv("api"),
    temperature=0,
   
)

response = llm.invoke("What is the meaning of life? in 10 words")
print(response.content)

Finding purpose, happiness, and fulfillment in one's existence and experiences.


In [14]:
import chromadb

client= chromadb.Client()
collection= client.create_collection(name="my_collection")



In [22]:
collection.add(
    documents=[
        "This document is about New York",
        "This document is about India"
    ],
    ids=['id1','id2']
)

/Users/yashshukla/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:41<00:00, 1.98MiB/s]


In [23]:
all_docs=collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This document is about New York',
  'This document is about India'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [25]:
documents= collection.get(ids=['id2'])
documents

{'ids': ['id2'],
 'embeddings': None,
 'documents': ['This document is about India'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [50]:
results=collection.query(
    query_texts=['Query is big apple'],
    n_results=2
)
results

{'ids': [['id1', 'id2']],
 'embeddings': None,
 'documents': [['This document is about New York',
   'This document is about India']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.750507116317749, 1.907548427581787]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [51]:
collection.delete(ids=all_docs['ids'])
collection.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [53]:
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids=['id3','id4'],
    metadatas=[
        {
            "url":"https://en.wikipedia.org/wiki/New_York_City"
        },
        {
            "url":"https://en.wikipedia.org/wiki/New_Delhi"
        }
    ]
)

In [55]:
documents= collection.get(ids=['id3'])
documents

{'ids': ['id3'],
 'embeddings': None,
 'documents': ['This document is about New York'],
 'uris': None,
 'data': None,
 'metadatas': [{'url': 'https://en.wikipedia.org/wiki/New_York_City'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [73]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
loader= WebBaseLoader("")
page_data= loader.load().pop().page_content
print(page_data)




Daily Search Trends
Recent searches
https://trends.google.com/trending/rss?geo=IN
		

chelsea f.c
100+

https://trends.google.com/trending/rss?geo=IN
			Thu, 7 Nov 2024 13:10:00 -0800
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSLE4loCcrQjXyEPNs3l5nYHomBmwbC6Ytf85P8XoLFjaN_BVdmtZEA3oQhJRI
OneFootball

Chelsea players “really truly believe” they can achieve this major step as 
soon as this week

https://onefootball.com/en/news/chelsea-players-really-truly-believe-they-can-achieve-this-major-step-as-soon-as-this-week-40277645
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSLE4loCcrQjXyEPNs3l5nYHomBmwbC6Ytf85P8XoLFjaN_BVdmtZEA3oQhJRI
OneFootball


Chelsea set up to break one bad habit which has hurt them so much in recent 
years

https://chelsea.news/2024/11/chelsea-bad-habit/
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcTY9REA96QWZ4Cs8JNmWXO_gpp2SaMui7ae9dhr0O4Uim_jLr47r4-PH9sxTxk
Chelsea News


Chelsea players “really truly believe” they can achieve this maj

In [150]:
from dotenv import load_dotenv
import os
import feedparser
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json
from pprint import pprint
import chromadb
from chromadb.utils import embedding_functions

In [ ]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Create or get collection
try:
    # Using OpenAI's embedding function (you can change this to other embedding functions if needed)
    embedding_function = embedding_functions.DefaultEmbeddingFunction()
    
    collection = chroma_client.create_collection(
        name="googleTrends",
        embedding_function=embedding_function,
        metadata={"description": "Google Trends daily trending topics"}
    )
    print("Created new collection: google_trends")
except ValueError:
    # Collection already exists
    collection = chroma_client.get_collection(
        name="google_trends",
        embedding_function=embedding_functions.DefaultEmbeddingFunction()
    )
    print("Using existing collection: google_trends")

In [153]:
def prepare_trends_for_chroma(trends):
    """
    Prepares trend data for ChromaDB storage
    """
    documents = []  # The actual content to embed
    metadatas = []  # Additional metadata for each document
    ids = []        # Unique IDs for each document
    
    for trend in trends:
        # Create a comprehensive document that includes all relevant information
        document = f"""
        Title: {trend['title']}
        Summary: {trend['summary']}
        Traffic: {trend['approximate_traffic']}
        
        Related News:
        {' '.join([f"{news['title']}: {news['snippet']}" for news in trend['news_items']])}
        """
        
        # Create metadata record
        metadata = {
            "title": trend['title'],
            "publish_date": trend['publish_date'],
            "traffic": trend['approximate_traffic'],
            "source": trend['source'],
            "timestamp": datetime.now().isoformat()
        }
        
        # Create unique ID using timestamp and title
        unique_id = f"trend_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{hash(trend['title'])}"
        
        documents.append(document)
        metadatas.append(metadata)
        ids.append(unique_id)
    
    return documents, metadatas, ids

In [ ]:
# Load environment variables
load_dotenv()

# Fetch trends (using your existing fetch_google_trends_detailed function)
trends = fetch_google_trends_detailed()

if not trends:
    trends = fetch_trends_detailed_backup()

if trends:
    # Prepare data for ChromaDB
    documents, metadatas, ids = prepare_trends_for_chroma(trends)
    
    # Add to ChromaDB
    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )
    
    print(f"\nSuccessfully stored {len(trends)} trends in ChromaDB")
    
    # Query example - fetch most recent trends
    results = collection.query(
        query_texts=["latest trends"],
        n_results=5,
        include=["metadatas", "documents"]
    )
    
    print("\nStored Trends (Recent Query Example):")
    for i, (metadata, document) in enumerate(zip(results['metadatas'][0], results['documents'][0]), 1):
        print(f"\n{i}. {metadata['title']}")
        print(f"Published: {metadata['publish_date']}")
        print(f"Traffic: {metadata['traffic']}")
        print("-" * 50)
else:
    print("Unable to fetch trending topics")

In [155]:
def query_trends(query_text, n_results=5):
    """
    Query the trends database
    """
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results,
        include=["metadatas", "documents"]
    )
    
    print(f"\nQuery Results for: '{query_text}'")
    for i, (metadata, document) in enumerate(zip(results['metadatas'][0], results['documents'][0]), 1):
        print(f"\n{i}. {metadata['title']}")
        print(f"Published: {metadata['publish_date']}")
        print(f"Traffic: {metadata['traffic']}")
        print(f"Document: {document[:200]}...") 
        print("-" * 50)

# Example usage:
# query_trends("technology")  # Search for technology-related trends
# query_trends("latest trends", n_results=3)  # Get latest 3 trends

In [ ]:
# Example queries to test the database
print("\nTesting different queries:")

print("\n1. Latest Trends:")
query_trends("latest trends", n_results=3)

print("\n2. Most Popular (by traffic):")
results = collection.query(
    query_texts=["high traffic trending topics"],
    n_results=3,
    include=["metadatas", "documents"]
)

print("\n3. Count total entries:")
print(f"Total trends stored: {collection.count()}")

In [ ]:
from dotenv import load_dotenv
import os
import chromadb
from chromadb.utils import embedding_functions
from langchain_groq import ChatGroq
import json
from operator import itemgetter

# Load environment variables
load_dotenv()

In [162]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_collection(
    name="googleTrends",
    embedding_function=embedding_functions.DefaultEmbeddingFunction()
)

# Initialize Groq
llm = ChatGroq(
   model="llama-3.1-70b-versatile",
    api_key=os.getenv("api"),
    temperature=0.7,
)

In [163]:
def get_top_trends_by_traffic(n=2):
    """
    Get top n trends sorted by traffic
    """
    # Get all items from collection
    results = collection.get(
        include=['metadatas', 'documents']
    )
    
    # Convert traffic strings to numbers for comparison
    trends_with_traffic = []
    for metadata, document in zip(results['metadatas'], results['documents']):
        # Extract numeric value from traffic string
        traffic_str = metadata['traffic'].replace(',', '').replace('K', '000').replace('+', '')
        try:
            traffic_num = float(traffic_str)
        except ValueError:
            traffic_num = 0
            
        trends_with_traffic.append({
            'metadata': metadata,
            'document': document,
            'traffic_num': traffic_num
        })
    
    # Sort by traffic and get top n
    sorted_trends = sorted(trends_with_traffic, 
                         key=itemgetter('traffic_num'), 
                         reverse=True)[:n]
    
    return sorted_trends

In [164]:
def generate_blog_for_trend(trend_data):
    """
    Generate a blog post about a trending topic using Groq
    """
    prompt = f"""
    Write an engaging and informative blog post about this trending topic:
    
    Title: {trend_data['metadata']['title']}
    Traffic: {trend_data['metadata']['traffic']}
    
    Context from news:
    {trend_data['document']}
    
    Write a comprehensive blog post that:
    1. Explains why this topic is trending
    2. Provides relevant context and background
    3. Analyzes its significance
    4. Includes key details from the news
    5. Concludes with future implications
    
    Make it engaging, well-structured, and around 400-500 words.
    """
    
    response = llm.invoke(prompt)
    return response.content

In [ ]:
# Get top 2 trending topics by traffic
top_trends = get_top_trends_by_traffic(2)

# Generate and display blogs for each trend
for i, trend in enumerate(top_trends, 1):
    print(f"\n{'='*80}")
    print(f"Blog #{i}: Trend - {trend['metadata']['title']}")
    print(f"Traffic: {trend['metadata']['traffic']}")
    print(f"{'='*80}\n")
    
    try:
        blog = generate_blog_for_trend(trend)
        print(blog)
        print(f"\n{'='*80}")
        
        # Optional: Save blogs to files
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'blog_{i}_{timestamp}.txt'
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"Trend: {trend['metadata']['title']}\n")
            f.write(f"Traffic: {trend['metadata']['traffic']}\n")
            f.write(f"Date: {trend['metadata']['publish_date']}\n\n")
            f.write(blog)
        print(f"\nBlog saved to {filename}")
        
    except Exception as e:
        print(f"Error generating blog for trend {i}: {e}")

In [189]:
import nest_asyncio
import asyncio
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, ContextTypes, MessageHandler, filters
from datetime import datetime
import logging
import schedule
import time
from threading import Thread
from dotenv import load_dotenv
import os
import sys


In [ ]:
# Enable nested event loops
nest_asyncio.apply()

# Setup logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Get bot token
BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
print("Loaded BOT_TOKEN:", BOT_TOKEN)

# Store user subscriptions in memory
subscribed_users = set()

In [ ]:
class TrendingBot:
    def __init__(self):
        self.application = None
        self.scheduler_thread = None
        self._running = False
        # Initialize schedule when bot starts
        schedule.clear()  # Clear any existing schedules
    
    def check_token(self):
        """Verify bot token is available and properly formatted."""
        if not BOT_TOKEN:
            logger.error("No bot token found! Make sure TELEGRAM_BOT_TOKEN is set in your .env file")
            sys.exit(1)
        
        if not BOT_TOKEN.strip():
            logger.error("Bot token is empty! Please set a valid token in your .env file")
            sys.exit(1)
        
        return BOT_TOKEN

    async def start(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Send a message when the command /start is issued."""
        user_id = update.message.from_user.id
        user_name = update.message.from_user.first_name
        
        keyboard = [
            [InlineKeyboardButton("📈 Current Trends", callback_data='current_trends')],
            [InlineKeyboardButton("📚 Latest Blogs", callback_data='latest_blogs')],
            [InlineKeyboardButton("🔍 Search Past Trends", callback_data='search')],
            [InlineKeyboardButton("📫 Subscribe to Daily Updates", callback_data='subscribe')]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        
        welcome_message = (
            f'Welcome {user_name} to the Trending Topics Bot! 🚀\n\n'
            'Choose an option below:'
        )
        
        logger.info(f"New user started bot: {user_id} ({user_name})")
        await update.message.reply_text(welcome_message, reply_markup=reply_markup)

    async def button_handler(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Handle button presses."""
        try:
            query = update.callback_query
            await query.answer()
            
            user_id = query.from_user.id
            user_name = query.from_user.first_name
            logger.info(f"Button press from user {user_id} ({user_name}): {query.data}")

            if query.data == 'current_trends':
                # Placeholder response for current trends
                await query.message.reply_text(
                    "📈 Here are today's top trending topics:\n\n"
                    "1. AI Development\n"
                    "2. Space Exploration\n"
                    "3. Climate Technology\n"
                    "\nNote: This is placeholder data"
                )

            elif query.data == 'latest_blogs':
                # Placeholder response for latest blogs
                await query.message.reply_text(
                    "📚 Latest blog posts:\n\n"
                    "1. Understanding AI Development\n"
                    "   A deep dive into current trends...\n\n"
                    "2. Space Tech Updates\n"
                    "   Latest developments in space...\n\n"
                    "Note: This is placeholder data"
                )

            elif query.data == 'search':
                await query.message.reply_text(
                    "🔍 Send me a keyword to search through past trends\n"
                    "Example: 'technology' or 'sports'"
                )
                context.user_data['expecting_search'] = True
                logger.info(f"User {user_id} initiated search mode")

            elif query.data == 'subscribe':
                user_id = query.from_user.id
                if user_id in subscribed_users:
                    await query.message.reply_text(
                        "You're already subscribed! ✅\n"
                        "You'll receive daily updates at 9:00 AM."
                    )
                else:
                    subscribed_users.add(user_id)
                    logger.info(f"New subscription from user {user_id} ({user_name})")
                    await query.message.reply_text(
                        "You're now subscribed to daily trending updates! 📫\n"
                        "You'll receive updates every day at 9:00 AM.\n\n"
                        "You can unsubscribe anytime by typing /unsubscribe"
                    )
        except Exception as e:
            logger.error(f"Error in button_handler: {e}")
            await query.message.reply_text(
                "Sorry, something went wrong while processing your request. "
                "Please try again later or contact support if the issue persists."
            )

    async def handle_search(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Handle search queries."""
        try:
            if context.user_data.get('expecting_search'):
                query_text = update.message.text
                user_id = update.message.from_user.id
                logger.info(f"Search query from user {user_id}: {query_text}")
                
                # Placeholder search response
                await update.message.reply_text(
                    f"🔍 Search results for '{query_text}':\n\n"
                    "1. Trending topic 1\n"
                    "   Relevance: High\n"
                    "   Date: 2024-01-01\n\n"
                    "2. Trending topic 2\n"
                    "   Relevance: Medium\n"
                    "   Date: 2024-01-02\n\n"
                    "Note: This is placeholder data"
                )
                context.user_data['expecting_search'] = False
        except Exception as e:
            logger.error(f"Error in handle_search: {e}")
            await update.message.reply_text(
                "Sorry, I encountered an error while searching. "
                "Please try again or contact support if the issue persists."
            )

    async def send_daily_update(self):
        """Send daily updates to subscribed users."""
        try:
            if not subscribed_users:
                logger.info("No subscribed users to send updates to")
                return

            message = (
                "📅 Today's Top Trending Topics:\n\n"
                "1. Topic One\n"
                "   Traffic: 50k views\n\n"
                "2. Topic Two\n"
                "   Traffic: 45k views\n\n"
                "3. Topic Three\n"
                "   Traffic: 40k views\n\n"
                "Note: This is placeholder data"
            )

            logger.info(f"Sending daily updates to {len(subscribed_users)} users")
            for user_id in subscribed_users.copy():  # Use copy to avoid modification during iteration
                try:
                    await self.application.bot.send_message(chat_id=user_id, text=message)
                    logger.info(f"Daily update sent to user {user_id}")
                except Exception as e:
                    logger.error(f"Failed to send update to user {user_id}: {e}")
                    # If the user has blocked the bot or deleted their account, remove them from subscriptions
                    if "Forbidden" in str(e):
                        subscribed_users.remove(user_id)
                        logger.info(f"Removed user {user_id} from subscriptions due to blocked/deleted account")
        except Exception as e:
            logger.error(f"Error in send_daily_update: {e}")

    def run_scheduler(self):
        """Run the scheduler in a separate thread."""
        try:
            # Schedule the daily update for 9:00 AM
            schedule.every().day.at("09:00").do(
                lambda: asyncio.run(self.send_daily_update())
            )
            
            logger.info("Scheduler started successfully")
            while self._running:
                schedule.run_pending()
                time.sleep(60)  # Check every minute
                
        except Exception as e:
            logger.error(f"Error in scheduler: {e}")
            if self._running:  # Only log if not shutting down
                logger.error("Scheduler stopped unexpectedly!")

    async def setup(self):
        """Setup the bot application."""
        try:
            # Verify token
            token = self.check_token()
            
            # Create application
            self.application = Application.builder().token(token).build()

            # Add handlers
            self.application.add_handler(CommandHandler("start", self.start))
            self.application.add_handler(CallbackQueryHandler(self.button_handler))
            self.application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, self.handle_search))

            # Initialize but don't start yet
            await self.application.initialize()
            
            logger.info("Bot setup completed successfully!")
            
        except Exception as e:
            logger.error(f"Error in setup: {e}")
            raise e

    async def start_polling(self):
        """Start the bot polling."""
        if not self.application:
            await self.setup()

        try:
            self._running = True
            # Start the scheduler in a separate thread
            self.scheduler_thread = Thread(target=self.run_scheduler, daemon=True)
            self.scheduler_thread.start()
            
            logger.info("Starting bot polling...")
            # Run the application polling directly
            await self.application.start()
            await self.application.run_polling(drop_pending_updates=True)
            
        except Exception as e:
            logger.error(f"Error while polling: {e}")
        finally:
            self._running = False
            await self.application.stop()
            logger.info("Bot stopped successfully")

def run_bot():
    """Run the bot."""
    bot = TrendingBot()
    loop = asyncio.get_event_loop()
    try:
        logger.info("Starting Trending Topics Bot...")
        loop.run_until_complete(bot.start_polling())
    except KeyboardInterrupt:
        logger.info("Bot stopped by user")
    except Exception as e:
        logger.error(f"Bot stopped due to error: {e}")
    finally:
        if not loop.is_running():  # Avoid closing a running event loop
            loop.close()
        logger.info("Bot shutdown complete")


async def main():
    try:
        # Start bot setup here
        await application.run_polling()
    except Exception as e:
        print("Error during polling:", e)
    finally:
        # Make sure to clean up resources
        if not application.is_running:
            print("Application stopped successfully.")

if __name__ == "__main__":
    asyncio.run(main())

2024-11-08 15:55:44,503 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7819747835:AAHDf6Et1TCC5YGJYJldbCcMonmfFTMcx8Q/getMe "HTTP/1.1 200 OK"
2024-11-08 15:55:44,505 - telegram.ext.Application - INFO - Application started
2024-11-08 15:55:44,731 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7819747835:AAHDf6Et1TCC5YGJYJldbCcMonmfFTMcx8Q/deleteWebhook "HTTP/1.1 200 OK"
2024-11-08 15:55:45,629 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7819747835:AAHDf6Et1TCC5YGJYJldbCcMonmfFTMcx8Q/getUpdates "HTTP/1.1 200 OK"
2024-11-08 15:55:45,631 - telegram.ext.Application - INFO - Application is stopping. This might take a moment.
2024-11-08 15:55:45,632 - telegram.ext.Application - INFO - Application.stop() complete
2024-11-08 15:55:45,633 - __main__ - ERROR - Bot stopped due to error: Cannot close a running event loop
2024-11-08 15:55:45,634 - __main__ - INFO - Bot shutdown complete
